In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from helpers import BRASIL_UFS ,get_uf_df

dfs = [get_uf_df(uf) for uf in BRASIL_UFS]

In [6]:
big_df = pd.concat(dfs)

In [7]:
print(len(big_df))
big_df.head()

1846924


,SG_UF,NM_MUNICIPIO,CD_MUNICIPIO,NR_PARTIDO,QT_VOTOS
0,RO,GUAJARÁ-MIRIM,19,-1,1
1,RO,GUAJARÁ-MIRIM,19,13,59
2,RO,GUAJARÁ-MIRIM,19,22,126
3,RO,GUAJARÁ-MIRIM,19,-1,4
8,RO,GUAJARÁ-MIRIM,19,-1,1


In [9]:
def get_pp_lula(x,df):
    try:
        votos_por_municipio = df.groupby(by=["NM_MUNICIPIO", "NR_PARTIDO"]).agg(
            quantidade=("QT_VOTOS", "sum")
        )
        votos_lula = votos_por_municipio["quantidade"][x["NM_MUNICIPIO"]][13]
        votos_bolsonaro = votos_por_municipio["quantidade"][x["NM_MUNICIPIO"]][22]
        total_votos_validos = votos_lula + votos_bolsonaro
        return votos_lula / total_votos_validos , total_votos_validos
    except:
        print(x["NM_MUNICIPIO"], "não encontrado no dataframe de votos por municipio.")
        return 0.5


def get_pp_municipio(df):
    ndf = df.drop_duplicates(subset=["NM_MUNICIPIO"])
    ndf["PP_LULA"] = ndf.apply(lambda x: get_pp_lula(x, df)[0], axis=1)
    ndf["QT_VOTOS"] = ndf.apply(lambda x: get_pp_lula(x, df)[1], axis=1)
    return ndf.drop('NR_PARTIDO', axis=1)

pp_df = get_pp_municipio(big_df)
pp_df

/tmp/ipykernel_9694/22180300.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf["PP_LULA"] = ndf.apply(lambda x: get_pp_lula(x, df)[0], axis=1)
/tmp/ipykernel_9694/22180300.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf["QT_VOTOS"] = ndf.apply(lambda x: get_pp_lula(x, df)[1], axis=1)


,SG_UF,NM_MUNICIPIO,CD_MUNICIPIO,QT_VOTOS,PP_LULA
0,RO,GUAJARÁ-MIRIM,19,19839,0.344675
301,RO,NOVA MAMORÉ,434,13123,0.220757
1250,RO,PORTO VELHO,35,261935,0.353660
3607,RO,ITAPUÃ DO OESTE,493,4378,0.375057
4163,RO,JI-PARANÁ,51,68126,0.245295
...,...,...,...,...,...
51990,GO,SÃO JOÃO D'ALIANÇA,95974,5305,0.509896
52093,GO,COLINAS DO SUL,92126,2314,0.671132
52415,GO,OURO VERDE DE GOIÁS,95052,2912,0.406593
53009,GO,CAMPO LIMPO DE GOIÁS,93580,4701,0.359073


In [10]:
pp_df['SG_UF'].value_counts()

MG    835
SP    620
RS    470
BA    396
PR    363
SC    260
GO    228
PI    223
MA    215
PB    204
CE    180
PE    168
RN    163
PA    144
TO    137
MT    132
AL     93
RJ     86
ES     75
MS     70
SE     67
AM     62
RO     52
AC     22
AP     16
RR     15
DF      1
Name: SG_UF, dtype: int64

In [ ]:
pp_df.to_csv('pp_lula_by_municipio.csv', index=False)